# H3: Amount vs Amount_USD

**H3 (Tech):** Конвертация сумм в USD улучшает стабильность модели и переносимость порогов.


In [1]:
import sys
from pathlib import Path
ROOT = Path().resolve()
if not (ROOT/'src').exists(): ROOT = ROOT.parent
sys.path.insert(0, str(ROOT))
print('Project root:', ROOT)


Project root: /Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025


In [2]:
import pandas as pd, numpy as np
from pathlib import Path
from sklearn.pipeline import Pipeline
from src.data import load_transactions, load_fx
from src.currency import convert_to_usd
from src.features import unpack_last_hour_activity, add_basic_time_features
from src.validation import split_time_aware
from src.pipeline import build_preprocessor, build_logreg
from src.eval import eval_pack

DATA=Path('../data'); TX=DATA/'transaction_fraud_data.parquet'; FX=DATA/'historical_currency_exchange.parquet'
df_raw = load_transactions(TX); fx = load_fx(FX)
df_usd = convert_to_usd(df_raw, fx); df_usd = unpack_last_hour_activity(df_usd); df_usd = add_basic_time_features(df_usd)

def train_eval(df_in, use_usd=True):
    cols = df_in.columns.tolist()
    keep = [c for c in cols if c != ('amount_usd' if not use_usd else 'amount')]
    d = df_in[keep].copy()
    train, test = split_time_aware(d)
    y_tr, y_te = train['is_fraud'].astype(int), test['is_fraud'].astype(int)
    X_tr, X_te = train.drop(columns=['is_fraud']), test.drop(columns=['is_fraud'])
    pipe = Pipeline([('prep', build_preprocessor(X_tr)), ('clf', build_logreg())]).fit(X_tr, y_tr)
    p = pipe.predict_proba(X_te)[:,1]
    return y_te, p

yA, pA = train_eval(df_usd, use_usd=False)  # amount
yB, pB = train_eval(df_usd, use_usd=True)   # amount_usd

/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: overflow encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: invalid value encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ wei

In [3]:
print(f'amount: {eval_pack(yA, pA)}')
print(f'amount_usd: { eval_pack(yB, pB)}')

amount: {'roc_auc': 0.5001122214983779, 'pr_auc': 0.1980998640895631, 'threshold_at_precision': 0.9, 'thr_value': None, 'recall_at_precision': None, 'precision_achieved': None}
amount_usd: {'roc_auc': 0.5001122265735541, 'pr_auc': 0.19809986674291455, 'threshold_at_precision': 0.9, 'thr_value': None, 'recall_at_precision': None, 'precision_achieved': None}


### Выводы по H3 (нормализация сумм в USD)

**Факт сравнения**
- Качество моделей на `amount` и `amount_usd` практически идентично:
  - `amount`: ROC-AUC ≈ **0.5001**, PR-AUC ≈ **0.19810**, порог при Precision≥0.90 — **не найден**.
  - `amount_usd`: ROC-AUC ≈ **0.5001**, PR-AUC ≈ **0.19810**, порог при Precision≥0.90 — **не найден**.
- ∆PR-AUC пренебрежимо мала (на уровне 1e-6): нормализация **не повлияла** на качество.

**Интерпретация**
- На текущем срезе данных нормализация валют **не даёт измеримого эффекта**. Возможные причины:
  1. Преобладание транзакций в **USD** → `amount ≈ amount_usd`.
  2. **Неверная ориентация курса** (деление вместо умножения, или наоборот) → фактически не меняем масштаб.
  3. **Пробелы в FX-join** (много `rate_to_currency = NaN` с последующей импутацией) → сигнал по сумме «схлопнут».
  4. Несостыковка по **дате/часовому поясу** (курс берётся на соседнюю дату).

**Рекомендации на следующую итерацию**
1. **Проверка данных FX/конверсии**
   - Доля не-USD транзакций и распределение `currency`.
   - Покрытие join: `% notnull(rate_to_currency)` по датам/валютам; при необходимости — `ffill` по дате.
   - Валидация формулы: если столбцы — *кол-во единиц валюты за 1 USD*, то `amount_usd = amount / rate`; если *USD за 1 единицу валюты*, то `amount_usd = amount * rate`.
2. **Оценка стабильноcти между странами**
   - PSI/JS для `amount` vs `amount_usd` по странам; цель — снижение дивергенции после нормализации.
   - Переносимость глобального порога: отклонение Precision/Recall при переносе порога из страны A в B (нормализация должна уменьшать разброс).
3. **Усиление «суммовых» фичей**
   - `log1p(amount_usd)`, каппинг хвостов (P99–P99.5).
   - Относительные признаки: отношение суммы к медиане трат клиента/карты/устройства, Z/MAD-скор.
   - Взаимодействия с CNP, cross-border, high-risk vendor.

**Итог по H3:** на данном прогоне гипотеза **не подтверждена**. Смысл нормализации появится, если существенна доля не-USD операций и корректно реализован FX-join; в противном случае эффект минимален.